In [1]:
import pandas as pd

In [5]:
data = pd.read_csv("dota2_chat_messages.csv", encoding="latin1")
data.head(10)

,match,time,slot,text
0,0,1005.12122,9,Ð»Ð°Ð´Ð½Ð¾ Ð³Ð³
1,0,1005.85442,9,Ð¸Ð·Ð¸
2,0,1008.65372,9,Ð¾Ð´
3,0,1010.51992,9,ÐµÐ±Ð°Ð½ÑÐ¹
4,0,1013.91912,9,Ð¼ÑÑÐ¾Ñ Ð½Ð° Ð²Ð¾Ð¹Ð´Ðµ
5,0,1800.31402,9,Ð¼ÑÑÐ¾Ñ
6,0,1801.71882,9,Ð½Ð° Ð²Ð¾Ð¹Ð´Ðµ
7,0,1802.98982,9,ÑÐµÐ¿Ð¾ÑÑ
8,0,1808.40822,9,100%
9,1,-131.14018,0,twitch.tv/rage_channel


In [6]:
data.groupby(['match', 'slot'])['text'].apply('.'.join).reset_index()

TypeError: sequence item 13: expected str instance, float found